In [1]:
import autokeras as ak
from pandas import read_csv
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
test = read_csv("NTI_data/test.csv")
train_target = read_csv("NTI_data/train_target.csv")
transactions_test = read_csv("NTI_data/transactions_test.csv")
transactions_train = read_csv("NTI_data/transactions_train.csv")

In [3]:
#Train
#Признаки, связанные с суммой покупки
agg_features=transactions_train.groupby('client_id')['amount_rur'].agg(['sum','mean','max']).reset_index()

#Количество в каждой категории
amount_of_purchase=transactions_train.groupby(['client_id','small_group'])['amount_rur'].count()
counts_train=amount_of_purchase.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
counts_train=counts_train.fillna(0)
counts_train.columns=['small_group_' + str(i) for i in counts_train.columns]

#Сумма в каждой категории
sum_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].sum()
sum_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
sum_train=sum_train.fillna(0)
sum_train.columns=['small_group_sum_'+str(i) for i in sum_train.columns]

#Минимум в каждой категории
min_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
min_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
min_train=min_train.fillna(0)
min_train.columns=['small_group_min_'+str(i) for i in min_train.columns]

#Максимум в каждой категории
max_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
max_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
max_train=max_train.fillna(0)
max_train.columns=['small_group_max_'+str(i) for i in max_train.columns]

#Средняя сумма в каждой категории
avg_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].mean()
avg_train=avg_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
avg_train=avg_train.fillna(0)
avg_train.columns=['small_group_avg_'+str(i) for i in avg_train.columns]

#Соединим все в одну таблицу
counts_train=pd.merge(sum_train,counts_train,on='client_id')
counts_train=pd.merge(avg_train,counts_train,on='client_id')
counts_train=pd.merge(min_train,counts_train,on='client_id')
counts_train=pd.merge(max_train,counts_train,on='client_id')
train=pd.merge(train_target,agg_features,on='client_id')
train=pd.merge(train,counts_train.reset_index(),on='client_id')

In [4]:
train_data = train.head(25000)
test_data = train.tail(5000)

In [7]:
model_finder = ak.StructuredDataRegressor()
X_train = train_data.drop('bins', 1)
y_train = train_data['bins']
X_test = test_data.drop('bins', 1)
y_test = test_data['bins']

In [ ]:
model_finder.fit(X_train, y_train)

Train for 625 steps, validate for 157 steps
Epoch 1/1000
625/625 [==============================] - ETA: 1:33:13 - loss: 2564410.2500 - mean_squared_error: 2564410.250 - ETA: 11:35 - loss: 1113522.1455 - mean_squared_error: 1113522.0000  - ETA: 6:09 - loss: 689648.3672 - mean_squared_error: 689648.2500  - ETA: 4:10 - loss: 634463.7823 - mean_squared_error: 634463.75 - ETA: 3:08 - loss: 512540.3970 - mean_squared_error: 512540.34 - ETA: 2:31 - loss: 430508.2662 - mean_squared_error: 430508.25 - ETA: 2:05 - loss: 437841.0719 - mean_squared_error: 437841.06 - ETA: 1:47 - loss: 628288.9330 - mean_squared_error: 628288.93 - ETA: 1:33 - loss: 575337.8860 - mean_squared_error: 575337.87 - ETA: 1:22 - loss: 542032.2002 - mean_squared_error: 542032.18 - ETA: 1:14 - loss: 523730.0725 - mean_squared_error: 523730.09 - ETA: 1:08 - loss: 490737.8499 - mean_squared_error: 490737.87 - ETA: 1:01 - loss: 479193.7396 - mean_squared_error: 479193.81 - ETA: 56s - loss: 447085.2304 - mean_squared_error: 44

625/625 [==============================] - ETA: 9:52 - loss: 48741.2969 - mean_squared_error: 48741.29 - ETA: 1:43 - loss: 41913.0387 - mean_squared_error: 41913.04 - ETA: 59s - loss: 33799.6803 - mean_squared_error: 33799.6836 - ETA: 42s - loss: 30620.7975 - mean_squared_error: 30620.798 - ETA: 33s - loss: 42483.7246 - mean_squared_error: 42483.726 - ETA: 29s - loss: 45772.6440 - mean_squared_error: 45772.644 - ETA: 26s - loss: 45279.1282 - mean_squared_error: 45279.132 - ETA: 23s - loss: 42370.5336 - mean_squared_error: 42370.539 - ETA: 22s - loss: 42378.9004 - mean_squared_error: 42378.906 - ETA: 20s - loss: 39128.9851 - mean_squared_error: 39128.988 - ETA: 19s - loss: 55874.5847 - mean_squared_error: 55874.582 - ETA: 18s - loss: 186462.9257 - mean_squared_error: 186462.937 - ETA: 16s - loss: 179933.1248 - mean_squared_error: 179933.140 - ETA: 15s - loss: 169560.8179 - mean_squared_error: 169560.828 - ETA: 15s - loss: 159303.6108 - mean_squared_error: 159303.625 - ETA: 14s - loss: 1

371/625 [================>.............] - ETA: 15:49 - loss: 3220.6980 - mean_squared_error: 3220.698 - ETA: 4:04 - loss: 3799.4979 - mean_squared_error: 3799.497 - ETA: 2:25 - loss: 2284583.3551 - mean_squared_error: 2284583.50 - ETA: 1:35 - loss: 1457998.3469 - mean_squared_error: 1457998.25 - ETA: 1:23 - loss: 1238494.8290 - mean_squared_error: 1238494.75 - ETA: 1:06 - loss: 965422.4878 - mean_squared_error: 965422.4375 - ETA: 57s - loss: 851334.1494 - mean_squared_error: 851334.1250 - ETA: 49s - loss: 713047.1790 - mean_squared_error: 713047.062 - ETA: 46s - loss: 658994.6487 - mean_squared_error: 658994.562 - ETA: 41s - loss: 573379.7103 - mean_squared_error: 573379.687 - ETA: 37s - loss: 508848.3830 - mean_squared_error: 508848.343 - ETA: 36s - loss: 482292.4554 - mean_squared_error: 482292.437 - ETA: 35s - loss: 457457.4633 - mean_squared_error: 457457.468 - ETA: 33s - loss: 427112.4083 - mean_squared_error: 427112.375 - ETA: 31s - loss: 402805.7540 - mean_squared_error: 402805

625/625 [==============================] - ETA: 6s - loss: 7460847.9693 - mean_squared_error: 7460846.00 - ETA: 6s - loss: 7401706.8449 - mean_squared_error: 7401705.00 - ETA: 6s - loss: 7362681.6456 - mean_squared_error: 7362679.50 - ETA: 6s - loss: 7304882.1084 - mean_squared_error: 7304879.50 - ETA: 6s - loss: 7285851.1917 - mean_squared_error: 7285849.00 - ETA: 6s - loss: 7210564.3930 - mean_squared_error: 7210562.00 - ETA: 6s - loss: 7173543.0121 - mean_squared_error: 7173540.50 - ETA: 5s - loss: 7118792.6978 - mean_squared_error: 7118790.00 - ETA: 5s - loss: 7082891.8806 - mean_squared_error: 7082889.00 - ETA: 5s - loss: 7047209.3525 - mean_squared_error: 7047206.00 - ETA: 5s - loss: 6982866.1116 - mean_squared_error: 6982862.50 - ETA: 5s - loss: 6930889.0864 - mean_squared_error: 6930885.50 - ETA: 5s - loss: 6896668.1348 - mean_squared_error: 6896664.50 - ETA: 5s - loss: 6862836.1271 - mean_squared_error: 6862833.00 - ETA: 5s - loss: 6829283.8692 - mean_squared_error: 6829280.50

361/625 [================>.............] - ETA: 13:55 - loss: 1827.9388 - mean_squared_error: 1827.938 - ETA: 4:48 - loss: 1509.5596 - mean_squared_error: 1509.559 - ETA: 2:09 - loss: 37136189.5996 - mean_squared_error: 37136188.00 - ETA: 1:33 - loss: 25995934.8219 - mean_squared_error: 25995934.00 - ETA: 1:20 - loss: 21663732.6742 - mean_squared_error: 21663732.00 - ETA: 1:06 - loss: 17333141.3780 - mean_squared_error: 17333142.00 - ETA: 1:00 - loss: 15294589.6969 - mean_squared_error: 15294589.00 - ETA: 55s - loss: 13684978.5156 - mean_squared_error: 13684979.0000 - ETA: 49s - loss: 11819723.9159 - mean_squared_error: 11819724.000 - ETA: 44s - loss: 10404584.9720 - mean_squared_error: 10404585.000 - ETA: 39s - loss: 8970531.2716 - mean_squared_error: 8970531.0000  - ETA: 36s - loss: 8130203.2806 - mean_squared_error: 8130203.500 - ETA: 35s - loss: 7652212.8953 - mean_squared_error: 7652213.000 - ETA: 32s - loss: 6847087.0949 - mean_squared_error: 6847087.500 - ETA: 30s - loss: 619544

...

625/625 [==============================] - ETA: 1s - loss: 1.2707 - mean_squared_error: 1.27 - ETA: 1s - loss: 1.2717 - mean_squared_error: 1.27 - ETA: 1s - loss: 1.2708 - mean_squared_error: 1.27 - ETA: 1s - loss: 1.2707 - mean_squared_error: 1.27 - ETA: 0s - loss: 1.2708 - mean_squared_error: 1.27 - ETA: 0s - loss: 1.2707 - mean_squared_error: 1.27 - ETA: 0s - loss: 1.2707 - mean_squared_error: 1.27 - ETA: 0s - loss: 1.2695 - mean_squared_error: 1.26 - ETA: 0s - loss: 1.2687 - mean_squared_error: 1.26 - ETA: 0s - loss: 1.2690 - mean_squared_error: 1.26 - ETA: 0s - loss: 1.2688 - mean_squared_error: 1.26 - ETA: 0s - loss: 1.2689 - mean_squared_error: 1.26 - ETA: 0s - loss: 1.2698 - mean_squared_error: 1.26 - ETA: 0s - loss: 1.2696 - mean_squared_error: 1.26 - ETA: 0s - loss: 1.2699 - mean_squared_error: 1.26 - ETA: 0s - loss: 1.2704 - mean_squared_error: 1.27 - ETA: 0s - loss: 1.2699 - mean_squared_error: 1.26 - ETA: 0s - loss: 1.2698 - mean_squared_error: 1.26 - ETA: 0s - loss: 1.270

504/625 [=======================>......] - ETA: 9:36 - loss: 0.9488 - mean_squared_error: 0.94 - ETA: 2:02 - loss: 1.2681 - mean_squared_error: 1.26 - ETA: 1:31 - loss: 1.2689 - mean_squared_error: 1.26 - ETA: 1:08 - loss: 1.2419 - mean_squared_error: 1.24 - ETA: 54s - loss: 1.3194 - mean_squared_error: 1.3194 - ETA: 43s - loss: 1.2959 - mean_squared_error: 1.295 - ETA: 38s - loss: 1.2657 - mean_squared_error: 1.265 - ETA: 33s - loss: 1.2872 - mean_squared_error: 1.287 - ETA: 31s - loss: 1.2902 - mean_squared_error: 1.290 - ETA: 29s - loss: 1.2800 - mean_squared_error: 1.280 - ETA: 28s - loss: 1.2877 - mean_squared_error: 1.287 - ETA: 26s - loss: 1.2881 - mean_squared_error: 1.288 - ETA: 24s - loss: 1.2813 - mean_squared_error: 1.281 - ETA: 23s - loss: 1.2757 - mean_squared_error: 1.275 - ETA: 22s - loss: 1.2830 - mean_squared_error: 1.283 - ETA: 22s - loss: 1.2763 - mean_squared_error: 1.276 - ETA: 21s - loss: 1.2781 - mean_squared_error: 1.278 - ETA: 20s - loss: 1.2658 - mean_squared

625/625 [==============================] - ETA: 2s - loss: 1.2469 - mean_squared_error: 1.24 - ETA: 2s - loss: 1.2466 - mean_squared_error: 1.24 - ETA: 2s - loss: 1.2466 - mean_squared_error: 1.24 - ETA: 2s - loss: 1.2471 - mean_squared_error: 1.24 - ETA: 2s - loss: 1.2470 - mean_squared_error: 1.24 - ETA: 2s - loss: 1.2471 - mean_squared_error: 1.24 - ETA: 2s - loss: 1.2475 - mean_squared_error: 1.24 - ETA: 1s - loss: 1.2473 - mean_squared_error: 1.24 - ETA: 1s - loss: 1.2476 - mean_squared_error: 1.24 - ETA: 1s - loss: 1.2477 - mean_squared_error: 1.24 - ETA: 1s - loss: 1.2482 - mean_squared_error: 1.24 - ETA: 1s - loss: 1.2490 - mean_squared_error: 1.24 - ETA: 1s - loss: 1.2499 - mean_squared_error: 1.24 - ETA: 1s - loss: 1.2500 - mean_squared_error: 1.25 - ETA: 1s - loss: 1.2500 - mean_squared_error: 1.25 - ETA: 1s - loss: 1.2500 - mean_squared_error: 1.25 - ETA: 1s - loss: 1.2507 - mean_squared_error: 1.25 - ETA: 1s - loss: 1.2505 - mean_squared_error: 1.25 - ETA: 1s - loss: 1.250

614/625 [============================>.] - ETA: 7:17 - loss: 0.9393 - mean_squared_error: 0.93 - ETA: 1:19 - loss: 1.2488 - mean_squared_error: 1.24 - ETA: 50s - loss: 1.2177 - mean_squared_error: 1.2177 - ETA: 38s - loss: 1.3097 - mean_squared_error: 1.309 - ETA: 31s - loss: 1.2750 - mean_squared_error: 1.275 - ETA: 27s - loss: 1.2593 - mean_squared_error: 1.259 - ETA: 24s - loss: 1.2661 - mean_squared_error: 1.266 - ETA: 21s - loss: 1.2642 - mean_squared_error: 1.264 - ETA: 20s - loss: 1.2631 - mean_squared_error: 1.263 - ETA: 18s - loss: 1.2644 - mean_squared_error: 1.264 - ETA: 17s - loss: 1.2630 - mean_squared_error: 1.263 - ETA: 16s - loss: 1.2686 - mean_squared_error: 1.268 - ETA: 16s - loss: 1.2659 - mean_squared_error: 1.265 - ETA: 15s - loss: 1.2529 - mean_squared_error: 1.252 - ETA: 14s - loss: 1.2522 - mean_squared_error: 1.252 - ETA: 14s - loss: 1.2494 - mean_squared_error: 1.249 - ETA: 13s - loss: 1.2605 - mean_squared_error: 1.260 - ETA: 13s - loss: 1.2572 - mean_squared

625/625 [==============================] - ETA: 0s - loss: 1.2447 - mean_squared_error: 1.24 - ETA: 0s - loss: 1.2453 - mean_squared_error: 1.24 - 23s 36ms/step - loss: 1.2451 - mean_squared_error: 1.2451 - val_loss: 1.2489 - val_mean_squared_error: 1.2465
Epoch 22/1000
401/625 [==================>...........] - ETA: 7:12 - loss: 0.9376 - mean_squared_error: 0.93 - ETA: 1:33 - loss: 1.2285 - mean_squared_error: 1.22 - ETA: 55s - loss: 1.2257 - mean_squared_error: 1.2257 - ETA: 40s - loss: 1.2907 - mean_squared_error: 1.290 - ETA: 33s - loss: 1.2736 - mean_squared_error: 1.273 - ETA: 28s - loss: 1.2425 - mean_squared_error: 1.242 - ETA: 24s - loss: 1.2689 - mean_squared_error: 1.268 - ETA: 22s - loss: 1.2556 - mean_squared_error: 1.255 - ETA: 20s - loss: 1.2664 - mean_squared_error: 1.266 - ETA: 19s - loss: 1.2627 - mean_squared_error: 1.262 - ETA: 18s - loss: 1.2660 - mean_squared_error: 1.266 - ETA: 17s - loss: 1.2648 - mean_squared_error: 1.264 - ETA: 16s - loss: 1.2586 - mean_square

In [ ]:
model_finder.evaluate(X_test, y_test)